In [7]:
!pip install tika
!pip install razdel
!pip install pymorphy2

     -------------------------------------- 55.5/55.5 kB 361.9 kB/s eta 0:00:00
     ---------------------------------------- 8.2/8.2 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f90120839f35568bde73340d12d285bcebd03e13c69c96855584ac025dc53dec
  Stored in directory: c:\users\265375\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [3]:
import tika
tika.initVM()

In [8]:
import os
import json
import re

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier

from razdel import tokenize
from pymorphy2 import MorphAnalyzer


# import fitz
# import docx2txt

# Чтение файлов

In [9]:
from tika import parser

def get_document_text(filepath):
    doc_type = filepath.split('.')[-1]
    if doc_type in ['doc', 'rtf', 'pdf', 'docx']:
        text = parser.from_file(filepath)['content']
    else:
        print(f'File extention must be either pdf, rtf, doc or docx, got {doc_type}')
    return text.strip()

In [13]:
datadir = 'C:/Users/265375/Desktop/Work/data/docs'

In [14]:
%%time

filename2text = {}

for filename in os.listdir(datadir):
    datapath = os.path.join(datadir, filename)
    try:
        filename2text[filename] = get_document_text(datapath)
    except:
        continue

2022-12-17 02:59:33,977 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to C:\Users\265375\AppData\Local\Temp\tika-server.jar.
2022-12-17 02:59:42,189 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to C:\Users\265375\AppData\Local\Temp\tika-server.jar.md5.
2022-12-17 02:59:44,199 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 02:59:44,200 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 02:59:45,371 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 02:59:45,372 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 02:59:46,528 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 02:59:46,528 [MainThread  ] [ERROR]  Failed to receive startup confirmation from start

2022-12-17 03:00:31,935 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:00:31,936 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:00:33,088 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:00:33,089 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:00:34,240 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:00:34,241 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:00:35,392 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:00:35,393 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:00:36,546 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:00:36,547 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:00:37,706 [MainThread  ] [ERROR]  Un

2022-12-17 03:01:21,720 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:01:22,874 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:01:22,875 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:01:24,029 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:01:24,030 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:01:25,184 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:01:25,185 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:01:26,339 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:01:26,340 [MainThread  ] [ERROR]  Failed to receive startup confirmation from startServer.
2022-12-17 03:01:27,494 [MainThread  ] [ERROR]  Unable to run java; is it installed?
2022-12-17 03:01:27,494 [MainThread  ] [ERROR]  Fa

Wall time: 2min 29s


In [14]:
data = pd.DataFrame({i: {'text': v, 'filename': k} for i, (k, v) in enumerate(filename2text.items())}).T
data.head()

,text,filename
0,ДОГОВОР № ______\n\nДОГОВОР\n\nг. Москва\n«___...,02682d726b725f95b9ee85f751c043d0.doc
1,ДОГОВОР \n\n№ ______ от «___» ___________ ____...,03a70fe60be9ecc2a63798a361fc9689.docx
2,ДОГОВОР КУПЛИ-ПРОДАЖИ КВАРТИРЫ\n\n\nДОГОВОР \n...,073a0d372820c3c2bffe9ba24a7ed7af.doc
3,Договор аренды земельного участка\n\n2\n5\n\nД...,084edc7dfc3db04e3a3c55c102f47bcb.doc
4,(Типовая форма) \n\nДОГОВОР КУПЛИ-ПРОДАЖИ №___...,086ad48895d2a73854b6151decc28800.pdf


In [15]:
with open('../data/xmas/classes.json', 'r', encoding='utf-8') as f:
    markup = json.load(f)

In [16]:
data['target'] = data['filename'].apply(lambda x: markup[x].split('/')[1])

In [17]:
data['lower_text'] = data['text'].apply(lambda x: x.lower())

In [18]:
le = LabelEncoder()
data['label'] = le.fit_transform(data['target'])

In [19]:
data.head()

,text,filename,target,lower_text,label
0,ДОГОВОР № ______\n\nДОГОВОР\n\nг. Москва\n«___...,02682d726b725f95b9ee85f751c043d0.doc,Договоры оказания услуг,договор № ______\n\nдоговор\n\nг. москва\n«___...,2
1,ДОГОВОР \n\n№ ______ от «___» ___________ ____...,03a70fe60be9ecc2a63798a361fc9689.docx,Договоры подряда,договор \n\n№ ______ от «___» ___________ ____...,3
2,ДОГОВОР КУПЛИ-ПРОДАЖИ КВАРТИРЫ\n\n\nДОГОВОР \n...,073a0d372820c3c2bffe9ba24a7ed7af.doc,Договоры купли-продажи,договор купли-продажи квартиры\n\n\nдоговор \n...,1
3,Договор аренды земельного участка\n\n2\n5\n\nД...,084edc7dfc3db04e3a3c55c102f47bcb.doc,Договоры аренды,договор аренды земельного участка\n\n2\n5\n\nд...,0
4,(Типовая форма) \n\nДОГОВОР КУПЛИ-ПРОДАЖИ №___...,086ad48895d2a73854b6151decc28800.pdf,Договоры купли-продажи,(типовая форма) \n\nдоговор купли-продажи №___...,1


In [15]:
data = pd.read_csv('C:/Users/265375/Desktop/Work/data/docs.csv', sep = '\t',index_col = 0)
texts = pd.read_csv('C:/Users/265375/Desktop/Work/data/data.csv', sep='\t',index_col = 0)

In [16]:
label_dict = {'Договоры аренды': 0, 
              'Договоры купли-продажи': 1, 
              'Договоры оказания услуг':2,
              'Договоры подряда': 3,
              'Договоры поставки': 4}

In [21]:
data['label'] = data.target.apply(lambda x: label_dict[x])
texts['label'] = texts.target.apply(lambda x: label_dict[x])
texts = texts[['text', 'label']]
data = data[['text', 'label']]
mega_data = pd.concat([data, texts], axis=0, ignore_index=True)

In [23]:
mega_data["text"]

0      ДОГОВОР АРЕНДЫ&NBSP;\nавтомобильного прицепа\n...
1      ДОГОВОР АРЕНДЫ\nбанковской ячейки\nг. ________...
2      ДОГОВОР АРЕНДЫ\nвагонов\nг. _______________\n«...
3      ДОГОВОР АРЕНДЫ&NBSP;\nвоздушного судна (вертол...
4      ДОГОВОР АРЕНДЫ\nгаража (в частной собственност...
                             ...                        
704    Страница 1 из 8 \n\n \n\n \n\nДОГОВОР КУПЛИ-ПР...
705    Договор поставки товара. Образец 2015 года\n\n...
706    Mechel Agreement\n\nФорма 5.3.\n\nПриложение №...
707    ФОРМА 5.4.                                    ...
708    Договор купли-продажи товаров\n\nPAGE  \n5\n\n...
Name: text, Length: 709, dtype: object

# Моделирование

In [18]:
rand_state = 21

In [38]:
pipe = Pipeline(steps=[('vect', CountVectorizer()),
                        ('est', LogisticRegression(random_state=rand_state,max_iter=100))])

In [20]:
skf = StratifiedKFold()

In [39]:
for train_index, test_index in skf.split(X=mega_data["text"].values, y=mega_data['label'].values):
    X_train, y_train = mega_data['text'].values[train_index], mega_data['label'].values[train_index]
    X_test, y_test = mega_data['text'].values[test_index], mega_data['label'].values[test_index]
    pipe = Pipeline(steps=[('vect', CountVectorizer()),
                            ('est', LogisticRegression(random_state=rand_state))])
    pipe.fit(X=X_train, y=y_train)
    y_pred = pipe.predict(X_test)
    print(classification_report(y_test, y_pred))

C:\Users\265375\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.97      0.97      0.97        34
           1       0.91      0.87      0.89        23
           2       0.82      0.92      0.87        49
           3       0.81      0.62      0.70        21
           4       0.93      0.93      0.93        15

    accuracy                           0.88       142
   macro avg       0.89      0.86      0.87       142
weighted avg       0.88      0.88      0.88       142



C:\Users\265375\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.92      0.97      0.94        34
           1       0.86      0.78      0.82        23
           2       0.85      0.81      0.83        48
           3       0.74      0.91      0.82        22
           4       0.83      0.67      0.74        15

    accuracy                           0.85       142
   macro avg       0.84      0.83      0.83       142
weighted avg       0.85      0.85      0.84       142



C:\Users\265375\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.97      0.97      0.97        34
           1       0.95      0.87      0.91        23
           2       0.85      0.98      0.91        48
           3       1.00      0.91      0.95        22
           4       1.00      0.80      0.89        15

    accuracy                           0.93       142
   macro avg       0.96      0.91      0.93       142
weighted avg       0.94      0.93      0.93       142



C:\Users\265375\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.89      0.97      0.93        33
           1       0.82      0.78      0.80        23
           2       0.92      0.90      0.91        49
           3       0.86      0.90      0.88        21
           4       0.86      0.75      0.80        16

    accuracy                           0.88       142
   macro avg       0.87      0.86      0.86       142
weighted avg       0.88      0.88      0.88       142

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        33
           1       0.91      0.91      0.91        23
           2       0.98      0.90      0.94        49
           3       0.90      0.90      0.90        21
           4       0.71      1.00      0.83        15

    accuracy                           0.92       141
   macro avg       0.90      0.93      0.91       141
weighted avg       0.93      0.92      0.92       141



C:\Users\265375\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Интерпретация

In [28]:
import eli5

In [40]:
vec=CountVectorizer()
est=LogisticRegression(random_state=rand_state)

pipe = make_pipeline(vec, est)
pipe.fit(X=X_train, y=y_train)

C:\Users\265375\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('logisticregression', LogisticRegression(random_state=21))])

In [41]:
expl = eli5.ipython.explain_prediction(est, X_test[0], vec=vec, 
                                       target_names=le.classes_, 
                                       targets=[le.classes_[-1]], top=5)

NameError: name 'le' is not defined

In [42]:
dict_expl = eli5.format_as_dict(eli5.ipython.explain_prediction(est, X_test[1], vec=vec,
                                target_names=le.classes_, targets=[le.classes_[1]], top=5))

NameError: name 'le' is not defined

# Обертка

In [15]:
pipe.fit(X=data['lower_text'].values, y=data['label'].values)

D:\Anaconda\envs\contract_type_clf\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vect', CountVectorizer()),
                ('est', LogisticRegression(random_state=21))])

In [16]:
import pickle

In [17]:
with open('baseline.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [1]:
import tika
tika.initVM()
from tika import parser
import pickle

In [2]:
class Predictor(object):
    def __init__(self, model_path: str):
        self.id2label = {0: 'Договоры аренды',
                         1: 'Договоры купли-продажи',
                         2: 'Договоры оказания услуг',
                         3: 'Договоры подряда',
                         4: 'Договоры поставки'}
        with open('baseline.pkl', 'rb') as f:
            self.model = pickle.load(f)
            
    def predict_from_file(self, filepath: str) -> str:
        text = Predictor._get_document_text(filepath)
        return self.id2label[self.model.predict([text])[0]]
        
    @staticmethod
    def _get_document_text(filepath: str) -> str:
        doc_type = filepath.split('.')[-1]
        if doc_type in ['doc', 'rtf', 'pdf', 'docx']:
            text = parser.from_file(filepath)['content']
        else:
            raise NotImplementedError(f'File extention must be either pdf, rtf, doc or docx, got {doc_type}')
        return text.strip().lower()

In [3]:
pred = Predictor('baseline.pkl')

In [4]:
pred.predict_from_file('../data/xmas/docs/02682d726b725f95b9ee85f751c043d0.doc')

'Договоры оказания услуг'